<a href="https://colab.research.google.com/github/wko1014/RL_Study/blob/main/notes/dpdpDynamic_Programming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import API
import numpy as np

In [ ]:
# Define a function to get the state coordinate.
def state_coord(state, action, grid_size):
  # Define action_code
  code = [(-1, 0), (1, 0), (0, -1), (0, 1)]
  
  state[0] += code[action][0]
  state[1] += code[action][1]
  
  # The agent cannot go outside from the grid world.
  if state[0] < 0:
    state[0] = 0
  elif state[0] > grid_size-1:
    state[0] = grid_size-1
    
  if state[1] < 0:
    state[1] = 0
  elif state[1] > grid_size-1:
    state[1] = grid_size-1
    
  return state[0], state[1]  

In [ ]:
# Define policy evaluation function
def policy_evaluation(grid_size, action, policy, iter_num, reward=-1, dis=1):
  # Initialize policy table
  value_table = np.zeros((grid_size, grid_size), dtype=float)
  
  # Iteration
  if iter_num == 0:
    print("Iteration: {} \n{}\n".format(iter_num, value_table))
    return value_table
  
  for iteration in range(iter_num):
    tmp = np.zeros((grid_size, grid_size), dtype=float)
    for i in range(grid_size):
      for j in range(grid_size):
        if i == j and ((i == 0) or (i == grid_size-1)):
          value_t = 0
        else:
          value_t = 0
          for act in action:
            i_, j_ = state_coord([i, j], act, grid_size)
            value = policy[i][j][act] * (reward + dis*value_table[i_][j_])
            value_t += value
        tmp[i][j] = round(value_t, grid_size-4)
    iteration += 1
    # Result print
    if (iteration % 10) != iter_num:
      if iteration > 100:
        if (iteration % 20) == 0:
          print("Iteration: {} \n{}\n".format(iteration, tmp))
      else:
        if (iteration % 10) == 0:
          print("Iteration: {} \n{}\n".format(iteration, tmp))
    else:
      print("Iteration: {} \n{}\n".format(iteration, tmp))

    value_table = tmp
    
  return tmp

In [ ]:
grid_size = 5
action = [0, 1, 2, 3] # Up, Down, Left, Right
policy = np.empty([grid_size, grid_size, len(action)], dtype=float)
for i in range(grid_size):
  for j in range(grid_size):
    for k in range(len(action)):
      if i==j and ((i==0) or (i==grid_size-1)):
        policy[i][j]=0.00
      else:
        policy[i][j]=0.25
        
policy[0][0] = [0] * len(action)
policy[grid_size-1][grid_size-1] = [0] * len(action)

In [ ]:
value = policy_evaluation(grid_size=grid_size, action=action, policy=policy, iter_num=100)

Iteration: 10 
[[ 0.  -6.3 -8.7 -9.5 -9.8]
 [-6.3 -8.  -9.1 -9.5 -9.6]
 [-8.7 -9.1 -9.3 -9.1 -8.7]
 [-9.5 -9.4 -9.1 -8.  -6.3]
 [-9.8 -9.5 -8.7 -6.3  0. ]]

Iteration: 20 
[[  0.  -10.4 -15.1 -17.  -17.6]
 [-10.4 -13.7 -15.9 -16.8 -17. ]
 [-15.  -15.9 -16.3 -15.9 -15. ]
 [-17.  -16.8 -15.9 -13.7 -10.4]
 [-17.6 -17.  -15.  -10.4   0. ]]

Iteration: 30 
[[  0.  -13.6 -19.8 -22.6 -23.6]
 [-13.6 -17.9 -21.  -22.4 -22.6]
 [-19.8 -21.  -21.5 -21.  -19.8]
 [-22.6 -22.3 -21.  -17.9 -13.6]
 [-23.5 -22.6 -19.8 -13.6   0. ]]

Iteration: 40 
[[  0.  -15.9 -23.4 -26.8 -28. ]
 [-15.9 -21.  -24.8 -26.4 -26.8]
 [-23.4 -24.8 -25.4 -24.8 -23.4]
 [-26.8 -26.4 -24.8 -21.  -15.9]
 [-28.  -26.8 -23.4 -15.9   0. ]]

Iteration: 50 
[[  0.  -17.6 -26.1 -30.  -31.4]
 [-17.6 -23.5 -27.6 -29.6 -30. ]
 [-26.1 -27.6 -28.4 -27.6 -26.1]
 [-30.  -29.6 -27.6 -23.4 -17.6]
 [-31.4 -30.  -26.1 -17.6   0. ]]

Iteration: 60 
[[  0.  -19.  -28.2 -32.3 -33.8]
 [-19.  -25.2 -29.8 -31.8 -32.3]
 [-28.2 -29.8 -30.6 -29.8 -28.2]
 

In [ ]:
# Define policy improvement function
def policy_improvement(grid_size, value, action, policy, reward=-1):
  action_id = ["U","D", "L", "R"] # Up, Down, Left, Right
  action_table = []
  
  # Calculate Q-function
  for i in range(grid_size):
    for j in range(grid_size):
      Q_list = []
      if i==j and ((i==0) or (i==grid_size-1)):
        action_table.append("G") # Goal
      else:
        for k in range(len(action)):
          i_, j_ = state_coord([i, j], k, grid_size)
          Q_list.append(value[i_][j_])
        max_action = [action_value for action_value, x in enumerate(Q_list) if x == max(Q_list)]
        
        # Policy update
        policy[i][j] = [0] * len(action) # Initialize Q list
        for y in max_action:
          policy[i][j][y] = (1/len(max_action))
          
        # Get action
        idx = np.argmax(policy[i][j])
        action_table.append(action_id[idx])
  action_table = np.asarray(action_table).reshape((grid_size, grid_size))
  
  print("Updated policy is :\n{}\n".format(policy))
  print("At each state, chosen action is: \n{}".format(action_table))
  
  return policy

In [ ]:
updated_policy = policy_improvement(grid_size=grid_size, value=value, action=action, policy=policy)

Updated policy is :
[[[0.   0.   0.   0.  ]
  [0.   0.   1.   0.  ]
  [0.   0.   1.   0.  ]
  [0.   0.   1.   0.  ]
  [0.   0.5  0.5  0.  ]]

 [[1.   0.   0.   0.  ]
  [0.5  0.   0.5  0.  ]
  [0.   0.   1.   0.  ]
  [0.   0.5  0.5  0.  ]
  [0.   1.   0.   0.  ]]

 [[1.   0.   0.   0.  ]
  [1.   0.   0.   0.  ]
  [0.25 0.25 0.25 0.25]
  [0.   1.   0.   0.  ]
  [0.   1.   0.   0.  ]]

 [[1.   0.   0.   0.  ]
  [0.5  0.   0.   0.5 ]
  [0.   0.   0.   1.  ]
  [0.   0.   0.   1.  ]
  [0.   1.   0.   0.  ]]

 [[0.5  0.   0.   0.5 ]
  [0.   0.   0.   1.  ]
  [0.   0.   0.   1.  ]
  [0.   0.   0.   1.  ]
  [0.   0.   0.   0.  ]]]

At each state, chosen action is: 
[['G' 'L' 'L' 'L' 'D']
 ['U' 'U' 'L' 'D' 'D']
 ['U' 'U' 'U' 'D' 'D']
 ['U' 'U' 'R' 'R' 'D']
 ['U' 'R' 'R' 'R' 'G']]


In [ ]:
# Define value iteration algorithm function
def value_iteration(grid_size, action, policy, iter_num, reward=-1, dis=1):
  # Initialize table
  value_table = np.zeros((grid_size, grid_size), dtype=float)
  
  # Iteration
  if iter_num == 0:
    print("Iteration: {} \n{}\n".format(iter_num, value_table))
    return value_table
  
  for iteration in range(iter_num):
    tmp = np.zeros((grid_size, grid_size), dtype=float)
    for i in range(grid_size):
      for j in range(grid_size):
        if i == j and ((i == 0) or (i == grid_size-1)):
          value_t = 0
        else:
          value_t_list = []
          for act in action:
            i_, j_ = state_coord([i, j], act, grid_size)
            value = (reward + dis*value_table[i_][j_])
            value_t_list.append(value)
          tmp[i][j] = max(value_t_list)
    iteration +=1
    # Result print
    if (iteration % 10) != iter_num:
      if iteration > 100:
        if (iteration % 20) == 0:
          print("Iteration: {} \n{}\n".format(iteration, tmp))
      else:
        if (iteration % 10) == 0:
          print("Iteration: {} \n{}\n".format(iteration, tmp))
    else:
      print("Iteration: {} \n{}\n".format(iteration, tmp))

    value_table = tmp
    
  return tmp

In [ ]:
grid_size = 5
action = [0, 1, 2, 3] # Up, Down, Left, Right
policy = np.empty([grid_size, grid_size, len(action)], dtype=float)
for i in range(grid_size):
  for j in range(grid_size):
    for k in range(len(action)):
      if i==j and ((i==0) or (i==grid_size-1)):
        policy[i][j]=0.00
      else:
        policy[i][j]=0.25
        
policy[0][0] = [0] * len(action)
policy[grid_size-1][grid_size-1] = [0] * len(action)

In [ ]:
value = value_iteration(grid_size=grid_size, action=action, policy=policy, iter_num=1)
value = value_iteration(grid_size=grid_size, action=action, policy=policy, iter_num=2)
value = value_iteration(grid_size=grid_size, action=action, policy=policy, iter_num=3)
value = value_iteration(grid_size=grid_size, action=action, policy=policy, iter_num=10)

Iteration: 1 
[[ 0. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1. -1.]
 [-1. -1. -1. -1.  0.]]

Iteration: 2 
[[ 0. -1. -2. -2. -2.]
 [-1. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -2.]
 [-2. -2. -2. -2. -1.]
 [-2. -2. -2. -1.  0.]]

Iteration: 3 
[[ 0. -1. -2. -3. -3.]
 [-1. -2. -3. -3. -3.]
 [-2. -3. -3. -3. -2.]
 [-3. -3. -3. -2. -1.]
 [-3. -3. -2. -1.  0.]]

Iteration: 10 
[[ 0. -1. -2. -3. -4.]
 [-1. -2. -3. -4. -3.]
 [-2. -3. -4. -3. -2.]
 [-3. -4. -3. -2. -1.]
 [-4. -3. -2. -1.  0.]]

